In [42]:
import os
import urllib.request
from datetime import datetime
import pandas as pd
import re
from io import StringIO

In [43]:
DATA_DIR = "data"
os.makedirs(DATA_DIR, exist_ok=True)

province_ids = range(1, 28)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

for pid in province_ids:
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={pid}&year1=1981&year2=2024&type=Mean"
    filename = os.path.join(DATA_DIR, f"province_{pid}_{timestamp}.csv")

    try:
        urllib.request.urlretrieve(url, filename)
        print(f"[OK] Завантажено: {filename}")
    except Exception as e:
        print(f"[Помилка] Область {pid}: {e}")



[OK] Завантажено: data\province_1_20250925_161736.csv
[OK] Завантажено: data\province_2_20250925_161736.csv
[OK] Завантажено: data\province_3_20250925_161736.csv
[OK] Завантажено: data\province_4_20250925_161736.csv
[OK] Завантажено: data\province_5_20250925_161736.csv
[OK] Завантажено: data\province_6_20250925_161736.csv
[OK] Завантажено: data\province_7_20250925_161736.csv
[OK] Завантажено: data\province_8_20250925_161736.csv
[OK] Завантажено: data\province_9_20250925_161736.csv
[OK] Завантажено: data\province_10_20250925_161736.csv
[OK] Завантажено: data\province_11_20250925_161736.csv
[OK] Завантажено: data\province_12_20250925_161736.csv
[OK] Завантажено: data\province_13_20250925_161736.csv
[OK] Завантажено: data\province_14_20250925_161736.csv
[OK] Завантажено: data\province_15_20250925_161736.csv
[OK] Завантажено: data\province_16_20250925_161736.csv
[OK] Завантажено: data\province_17_20250925_161736.csv
[OK] Завантажено: data\province_18_20250925_161736.csv
[OK] Завантажено: d

In [ ]:


DATA_DIR = "data"
all_data = []

for file in sorted(os.listdir(DATA_DIR)):
    if not file.endswith(".csv"):
        continue

    filepath = os.path.join(DATA_DIR, file)

    # TODO: use pd.read_csv instead with open(...
    with open(filepath, "r", encoding="utf-8", errors="replace") as f:
        lines = f.readlines()

    # TODO: use data cleaning skills instead
    header_idx = None
    header_line = ""
    for i, line in enumerate(lines[:200]):   
        if re.search(r"\byear\b", line, re.IGNORECASE) and re.search(r"\bweek\b", line, re.IGNORECASE):
            header_idx = i
            header_line = line
            break

    if header_idx is None:
        header_idx = 0
        header_line = lines[0] if lines else ""

    if "," in header_line:
        sep = ","
        use_whitespace = False
    elif ";" in header_line:
        sep = ";"
        use_whitespace = False
    elif "\t" in header_line:
        sep = "\t"
        use_whitespace = False
    else:
        sep = None
        use_whitespace = True

    data_lines = lines[header_idx:]
    if not data_lines:
        continue
    data_lines[0] = re.sub(r'^[#\s]+', '', data_lines[0])
    data_str = "".join(data_lines)

    if use_whitespace:
        df = pd.read_csv(StringIO(data_str),
                         delim_whitespace=True,
                         engine="python",
                         na_values=["-999", ""],
                         skipinitialspace=True)
    else:
        df = pd.read_csv(StringIO(data_str),
                         sep=sep,
                         engine="python",
                         na_values=["-999", ""],
                         skipinitialspace=True)

    df.columns = df.columns.str.strip()

    rename_map = {}
    for col in df.columns:
        key = re.sub(r'[^a-z0-9]', '', col.lower())
        if key == "year":
            rename_map[col] = "year"
        elif key == "week":
            rename_map[col] = "week"
        elif key == "smn":
            rename_map[col] = "SMN"
        elif key == "smt":
            rename_map[col] = "SMT"
        elif key == "vci":
            rename_map[col] = "VCI"
        elif key == "tci":
            rename_map[col] = "TCI"
        elif key == "vhi":
            rename_map[col] = "VHI"

    df = df.rename(columns=rename_map)
    
    keep_cols = [c for c in ["year", "week", "SMN", "SMT", "VCI", "TCI", "VHI"] if c in df.columns]
    if not keep_cols:
        continue

    df = df[keep_cols].copy()

    for col in keep_cols:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    m = re.search(r"province_(\d+)_", file)
    province_id = int(m.group(1)) if m else None
    df["province_id"] = province_id

    all_data.append(df)

if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
else:
    final_df = pd.DataFrame(columns=["year", "week", "SMN", "SMT", "VCI", "TCI", "VHI", "province_id"])

final_df = final_df.sort_values(["province_id", "year", "week"]).reset_index(drop=True)

print("Унікальних province_id:", final_df["province_id"].nunique())
print("Список областей:", sorted(final_df["province_id"].unique()))

# TODO: move to new logical block
province_names = {
    1: "Вінницька",
    2: "Волинська",
    3: "Дніпропетровська",
    4: "Донецька",
    5: "Житомирська",
    6: "Закарпатська",
    7: "Запорізька",
    8: "Івано-Франківська",
    9: "Київська",
    10: "Кіровоградська",
    11: "Луганська",
    12: "Львівська",
    13: "Миколаївська",
    14: "Одеська",
    15: "Полтавська",
    16: "Рівненська",
    17: "Сумська",
    18: "Тернопільська",
    19: "Харківська",
    20: "Херсонська",
    21: "Хмельницька",
    22: "Черкаська",
    23: "Чернівецька",
    24: "Чернігівська",
    25: "Республіка Крим",
    26: "м. Київ",
    27: "Севастополь"
}

final_df["province_name"] = final_df["province_id"].map(province_names)

# показати по 3 рядки для кожної області з назвами
final_df = final_df[final_df!=-1]
final_df = final_df.dropna()
print(final_df.head(50))
# display(final_df.groupby("province_name").head(1))

Унікальних province_id: 27
Список областей: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27)]
    year   week     SMN    SMT    VCI    TCI  province_id province_name
4    1.0  0.016  254.68   1.91  59.48  30.69            1     Вінницька
5    1.0  0.016  254.68   1.91  59.48  30.69            1     Вінницька
6    1.0  0.026  244.98  14.39  92.72  53.55            1     Вінницька
7    1.0  0.026  244.98  14.39  92.72  53.55            1     Вінницька
8    1.0  0.030  251.40  18.27  71.45  44.86            1     Вінницька
9    1.0  0.030  251.40  18.27  71.45  44.86            1     Вінницька
10   1.0  0.031  257.00  20.04  51.54  35.79            1     Вінницька
11   1.0  